In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import pandas as pd # data processing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv1D,Dropout,LSTM, Bidirectional
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from keras.layers import BatchNormalization, Dropout
from keras.layers import Dense, Activation
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
import torch
from torch.nn import Transformer
from sklearn.preprocessing import MinMaxScaler

In [2]:
FS = 30 # sampling rate
TOTAL_CLASSESS = 9
DATA_FOLDER = "./"
WINDOW_SIZE = 2 # seconds
OVERLAP_RATE = 0.25 * WINDOW_SIZE # overlap 50% of window size
standartscaler=MinMaxScaler()
def load_data(keypoint_csv, ann_csv):
    kp_df = pd.read_csv(keypoint_csv)
    kp_df = kp_df.loc[:, ~kp_df.columns.str.contains('^Unnamed')]

    ann_df = pd.read_csv(ann_csv)
    ann_df = ann_df.loc[:, ~ann_df.columns.str.contains('^Unnamed')]

    return kp_df, ann_df

In [3]:
def remove_redundant_kp(kp_df):
    kp_df = kp_df.loc[:, ~kp_df.columns.str.contains(
    'conf|left_knee|right_knee|left_ankle|right_ankle', regex=True)]
    return kp_df


In [4]:
SMOOTH_LEN = 3 # seconds

def smooth_kp(kp_col):
    zero_idx = np.flatnonzero(kp_col == 0)
    split_idx = np.split(zero_idx, np.flatnonzero(np.diff(zero_idx) > 1) + 1)
    for each_split_idx in split_idx:
        if len(each_split_idx) == 0 or each_split_idx[0] == 0 or each_split_idx[-1] == (len(kp_col) - 1) or len(each_split_idx) > SMOOTH_LEN*FS:
            continue
        xp = [each_split_idx[0] - 1, each_split_idx[-1] + 1]
        fp = kp_col[xp]
        interp_kp = np.interp(each_split_idx, xp, fp)
        kp_col[each_split_idx] = interp_kp
    return kp_col

In [5]:
def cal_angle(a, b, c):
    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return angle


def extract_joint_angles(kp_data, steps=2):
    # steps = 2 if kp_data is removed conf columns
    # steps = 3 if kp_data has conf columns
    left_elbow_shoulder_hip = np.asarray([cal_angle(kp_data[i, 7*steps:(7*steps+2)], kp_data[i, 5*steps:(5*steps+2)], kp_data[i, 11*steps:(11*steps+2)])
                                          for i in range(len(kp_data))])
    left_elbow_shoulder_hip = np.nan_to_num(left_elbow_shoulder_hip)


    right_elbow_shoulder_hip = np.asarray([cal_angle(kp_data[i, 8*steps:(8*steps+2)], kp_data[i, 6*steps:(6*steps+2)], kp_data[i, 12*steps:(12*steps+2)])
                                            for i in range(len(kp_data))])
    right_elbow_shoulder_hip = np.nan_to_num(right_elbow_shoulder_hip)


    left_wrist_elbow_shoulder = np.asarray([cal_angle(kp_data[i, 9*steps:(9*steps+2)], kp_data[i, 7*steps:(7*steps+2)], kp_data[i, 5*steps:(5*steps + 2)])
                                            for i in range(len(kp_data))])
    left_wrist_elbow_shoulder = np.nan_to_num(left_wrist_elbow_shoulder)


    right_wrist_elbow_shoulder = np.asarray([cal_angle(kp_data[i, 10*steps:(10*steps+2)], kp_data[i, 8*steps:(8*steps+2)], kp_data[i, 6*steps:(6*steps+2)])
                                              for i in range(len(kp_data))])
    right_wrist_elbow_shoulder = np.nan_to_num(right_wrist_elbow_shoulder)


    right_elbow_shoulder = np.asarray([cal_angle(kp_data[i, 8*steps:(8*steps+2)], kp_data[i, 6*steps:(6*steps+2)], kp_data[i, 5*steps:(5*steps+2)])
                                              for i in range(len(kp_data))])
    right_elbow_shoulder = np.nan_to_num(right_elbow_shoulder)


    left_elbow_shoulder = np.asarray([cal_angle(kp_data[i, 6*steps:(6*steps+2)], kp_data[i, 5*steps:(5*steps+2)], kp_data[i, 7*steps:(7*steps+2)])
                                              for i in range(len(kp_data))])
    left_elbow_shoulder = np.nan_to_num(left_elbow_shoulder)


    joint_angles = np.array([left_elbow_shoulder_hip,
                    right_elbow_shoulder_hip, left_wrist_elbow_shoulder, 
                             right_wrist_elbow_shoulder, right_elbow_shoulder,
                             left_elbow_shoulder]).T

    return joint_angles
def extract_velocity(kp_data):
    velocity = np.diff(kp_data, axis=0)
    return velocity
def moving(v):
    if v>0:
        return 1
    return 0
def is_moving(kp_data):
    velocity = np.diff(kp_data, axis=0)
    velocity=pd.DataFrame(velocity)
    for col in velocity.columns:
        velocity[col]=velocity[col].apply(lambda x: moving(x))
    velocity=np.array(velocity)
    return velocity
def extract_acceleration(kp_data):
    velocity = np.diff(kp_data, axis=0)
    acceleration = np.diff(velocity, axis=0)
    return acceleration
def segment(data, max_time, sub_window_size, stride_size):
    sub_windows = np.arange(sub_window_size)[None, :] + np.arange(0, max_time, stride_size)[:, None]

    row, col = np.where(sub_windows >= max_time)
    uniq_row = len(np.unique(row))

    if uniq_row > 0 and row[0] > 0:
        sub_windows = sub_windows[:-uniq_row, :]

    return data[sub_windows]
import scipy

def extract_feature(data, fs):
    mean_ft = np.mean(data, axis=0)
    std_ft = np.std(data, axis=0)
    max_ft = np.max(data, axis=0)
    min_ft = np.min(data, axis=0)
    var_ft = np.var(data, axis=0)
    med_ft = np.median(data, axis=0)
    sum_ft = np.sum(data, axis=0)
    std = np.std(data, axis=0)
    kurtosis=scipy.stats.kurtosis(data)
    skew=scipy.stats.skew(data)
    quantile25=np.quantile(data,0.25,axis=0)
    quantile75=np.quantile(data,0.75,axis=0)
    rms = np.sqrt(np.mean(data**2,axis=0))
    mean_absolute=np.mean(np.absolute(data),axis=0)
    features = np.array([mean_ft, std_ft, max_ft, min_ft,
                         var_ft, med_ft,sum_ft,std,kurtosis,skew,quantile25,quantile75
                         ,rms,mean_absolute]).T.flatten()
    features = np.nan_to_num(features)
    return features
def remove_redundant_kp(kp_df):
    kp_df = kp_df.loc[:, ~kp_df.columns.str.contains(
    'conf|left_knee|right_knee|left_ankle|right_ankle', regex=True)]
    return kp_df

In [6]:
def make_csv(USER_ID):
    USER_ID=USER_ID[:5]
    keypoint_csv = DATA_FOLDER + "train/{}_keypoint.csv".format(USER_ID)
    ann_csv = DATA_FOLDER + "/ann/{}_ann.csv".format(USER_ID)
    kp_df, ann_df = load_data(keypoint_csv, ann_csv)
    kp_df = remove_redundant_kp(kp_df)
    col=kp_df.columns
    kp_df=standartscaler.fit_transform(kp_df)
    kp_df=pd.DataFrame(kp_df,columns=col)
    feature_seg_feat=["left_elbow_shoulder_hip",
                    "right_elbow_shoulder_hip", "left_wrist_elbow_shoulder", 
                    "right_wrist_elbow_shoulder"," right_elbow_shoulder",
                    "left_elbow_shoulder"]
    
    
    feature=[]
    stat_feat=["mean_ft", "std_ft", "max_ft", "min_ft", "var_ft", "med_ft", "sum_ft",
               "std","kurtosis","skew","quantile 25",'quantile 75',"RMS","MAV"]


    for i in [e for e in kp_df.columns if e not in ('right_shoulder_y','right_shoulder_x')]:
        if i[-1]=='y':
            feat_name="right_shoulder "+i[:-2]+' dy'
            kp_df[feat_name]=kp_df['right_shoulder_y']-kp_df[i]
        else:
            feat_name="right_shoulder "+i[:-1]+' dx'
            kp_df[feat_name]=kp_df['right_shoulder_x']-kp_df[i]
    kp_df["center_x"]=kp_df.loc[:,kp_df.columns.str.contains("_x")].apply(np.mean)
    kp_df["center_y"]=kp_df.loc[:,kp_df.columns.str.contains("_y")].apply(np.mean)
    
    norm_feat=kp_df.columns
    vel_feat=[f"velo{x}" for x in norm_feat]
    acc_feat=[f"acc{x}" for x in norm_feat]
    moving_feat=[f"moving{x}" for x in norm_feat]
    for feat in feature_seg_feat:
        for f in stat_feat:
            feature.append("".join([feat," ",f]))

    for feat in norm_feat:
        for f in stat_feat:
            feature.append("".join([feat," ",f]))

    for feat in vel_feat:
        for f in stat_feat:
            feature.append("".join([feat," ",f]))
    for feat in acc_feat:
        for f in stat_feat:
            feature.append("".join([feat," ",f]))
    for feat in moving_feat:
        for f in stat_feat:
            feature.append("".join([feat," ",f]))
    for i in range(len(kp_df.columns)):
        kp_df.iloc[:, i] = smooth_kp(np.array(kp_df.iloc[:, i]))

    # Processing training data
    all_feature = []
    all_label = []

    for i in range(len(ann_df)):
        seg = kp_df.loc[int(ann_df['start_time'][i]*FS): int(ann_df['stop_time'][i]*FS)]
        seg_label = ann_df["annotation"].iloc[i]
        if len(seg) > 0 and (len(seg) >= WINDOW_SIZE * FS):
            # Calculate joint angles from keypoint data
            joint_angles = extract_joint_angles(np.array(seg))

            # Segment keypoint data and joint angles by WINDOW_SIZE and OVERLAP_RATE
            ws_seg = segment(np.array(seg), max_time=len(seg), sub_window_size=WINDOW_SIZE * FS, stride_size=int((WINDOW_SIZE - OVERLAP_RATE) * FS))
            joint_angles_seg = segment(joint_angles, max_time=len(seg), sub_window_size=WINDOW_SIZE * FS,
                                            stride_size=int((WINDOW_SIZE - OVERLAP_RATE) * FS))

            # Calculate velocity from each segment of keypoint data
            velocity_seg = [extract_velocity(ws_seg[i]) for i in range(len(ws_seg))]

            # Calculate acceleration from each segment of keypoint data
            acceleration_seg = [extract_acceleration(ws_seg[i]) for i in range(len(ws_seg))]
            
            # Calculate movement from each segment of keypoint data
            moving_seg = [is_moving(ws_seg[i]) for i in range(len(ws_seg))]
            # Calculate features from each segment of keypoint data, joint angles and velocity
            feature_seg = [extract_feature(ws_seg[i], FS) for i in range(len(ws_seg))]
            feature_joint_angles_seg = [extract_feature(joint_angles_seg[i], FS) for i in
                                      range(len(joint_angles_seg))]
            feature_velocity_seg = [extract_feature(extract_velocity(ws_seg[i]), FS) for i in range(len(ws_seg))]
            feature_acceleration_seg = [extract_feature(extract_acceleration(ws_seg[i]), FS) for i in range(len(ws_seg))]
            feature_moving_seg = [extract_feature(is_moving(ws_seg[i]), FS) for i in range(len(ws_seg))]
            # Concatenate all features
            feature_segs = np.concatenate([feature_seg, feature_joint_angles_seg, feature_velocity_seg,feature_acceleration_seg,feature_moving_seg], axis=1)

            all_feature.extend(feature_segs)
            all_label.extend([int(seg_label)]*len(ws_seg))

    df=pd.DataFrame(all_feature,columns=feature)
    b=list(all_label[1:])
    b.insert(0,0)
    df['previous_label']=b
    
    df['label'] = all_label
    testset=[x[:3] for x in os.listdir('./test_set_df')]
    if USER_ID[:3] not in testset:
        name=f"./training_set_df_acc/{USER_ID}.csv"
    else:
        name=f"./test_set_df_acc/{USER_ID}.csv"
    df.to_csv(name,index=False)
#     print(df)
    return df

In [7]:
make_csv('S08T1.csv')

/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_68662/3156830331.py:5: RuntimeWarning: invalid value encountered in scalar divide
  cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_68662/3156830331.py:6: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)
/Users/khangphan/anaconda3/lib/python3.10/site-packages/numpy/lib/shape_base.py:402: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  buff[ind] = asanyarray(func1d(inarr_view[ind], *args, **kwargs))


(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)


/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_68662/3156830331.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurtosis=scipy.stats.kurtosis(data)
/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_68662/3156830331.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skew=scipy.stats.skew(data)


(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(

,left_elbow_shoulder_hip mean_ft,left_elbow_shoulder_hip std_ft,left_elbow_shoulder_hip max_ft,left_elbow_shoulder_hip min_ft,left_elbow_shoulder_hip var_ft,left_elbow_shoulder_hip med_ft,left_elbow_shoulder_hip sum_ft,left_elbow_shoulder_hip std,left_elbow_shoulder_hip kurtosis,left_elbow_shoulder_hip skew,...,movingcenter_y sum_ft,movingcenter_y std,movingcenter_y kurtosis,movingcenter_y skew,movingcenter_y quantile 25,movingcenter_y quantile 75,movingcenter_y RMS,movingcenter_y MAV,previous_label,label
0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,8
1,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,8
2,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,8
3,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,8
4,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,7
195,0.019619,0.105654,0.591090,0.0,0.011163,0.0,1.177152,0.105654,25.036708,5.199627,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,7
196,0.200230,0.294568,0.673527,0.0,0.086771,0.0,12.013798,0.294568,-1.350113,0.797730,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,7
197,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,8


In [8]:
IDS=os.listdir("./train")
for ID in IDS:
    print(ID)
    make_csv(ID)

N04T1_keypoint.csv
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)


/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_68662/3156830331.py:5: RuntimeWarning: invalid value encountered in scalar divide
  cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
/Users/khangphan/anaconda3/lib/python3.10/site-packages/numpy/lib/shape_base.py:402: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  buff[ind] = asanyarray(func1d(inarr_view[ind], *args, **kwargs))


(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)


/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_68662/3156830331.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurtosis=scipy.stats.kurtosis(data)
/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_68662/3156830331.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skew=scipy.stats.skew(data)


(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)


...

(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)


/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_68662/3156830331.py:89: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurtosis=scipy.stats.kurtosis(data)
/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_68662/3156830331.py:90: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skew=scipy.stats.skew(data)


(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)


/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_68662/3156830331.py:6: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)


(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(59, 52)
(